In [ ]:
import pandas as pd
import re
from collections import Counter
import gensim
import heapq
from operator import itemgetter
from multiprocessing import Pool

In [ ]:
#list of punctuations
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
#remove punctuation from text
def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

In [ ]:
#remove number from text
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
#dictionary of contractions
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict) #get key-value pair from dict of contractions 

#remove contractions from text
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

In [ ]:
#convert to lowercase
def lowercase(x):
  x=str(x)
  return x.lower()

In [ ]:
train=pd.read_csv("/content/drive/My Drive/minor/english_dataset/english_dataset.tsv",sep='\t') #traing data
test=pd.read_csv("/content/drive/My Drive/minor/english_dataset/hasoc2019_en_test-2919.tsv",sep='\t') #testing data

In [ ]:
train['text']=train['text'].apply(clean_text) #remove punctuations
test['text']=test['text'].apply(clean_text) #remove punctuations
train['text']=train['text'].apply(clean_numbers) #remove numbers
test['text']=test['text'].apply(clean_numbers) #remove numbers
train['text']=train['text'].apply(replace_contractions) #remove contractions
test['text']=test['text'].apply(replace_contractions) #remove contractions
train['text']=train['text'].apply(lowercase) #convert to lowercase
test['text']=test['text'].apply(lowercase) #convert to lowercase

In [ ]:
train.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,dhonikeepstheglove watch sports minister kire...,NOT,NONE,NONE
1,hasoc_en_2,@politico no we should remember very clearly t...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,corbyn is too politically intellectual for bor...,NOT,NONE,NONE
4,hasoc_en_5,all the best to teamindia for another swimming...,NOT,NONE,NONE


In [ ]:
train=train.dropna() #remove missing values from training data
train=train.reset_index(drop=True)
test=test.dropna() #remove missing values from testing data
test=test.reset_index(drop=True)

In [ ]:
train['task_1']=train['task_1'].map({'HOF':0,'NOT':1}) #map label strings to numbers

In [ ]:
test['task_1']=test['task_1'].map({'HOF':0,'NOT':1})  #map label strings to numbers

In [ ]:
train.to_csv("/content/drive/My Drive/minor/english_dataset/eng1train.csv",index=False) #saving a seperate training dataset for sub-task 1
test.to_csv("/content/drive/My Drive/minor/english_dataset/eng1test.csv",index=False) #saving a seperate testing dataset for sub-task 1

In [ ]:
train=train[train.task_2!='NONE'] #remove rows where task_2=NONE
test=test[test.task_2!='NONE']  #remove rows where task_2=NONE
train['task_2']=train['task_2'].map({'PRFN':0,'HATE':1,'OFFN':2})   #map label strings to numbers
test['task_2']=test['task_2'].map({'PRFN':0,'HATE':1,'OFFN':2})     #map label strings to numbers
train['task_3']=train['task_3'].map({'UNT':0,'TIN':1})              #map label strings to numbers
test['task_3']=test['task_3'].map({'UNT':0,'TIN':1})                #map label strings to numbers
train.to_csv("/content/drive/My Drive/minor/english_dataset/eng23train.csv",index=False)  #saving a seperate training dataset for sub-task 2 and 3
test.to_csv("/content/drive/My Drive/minor/english_dataset/eng23test.csv",index=False)    #saving a seperate testing dataset for sub-task 2 and 3